## Preprocessing 01 (Working with Sampling)
#### 1. Crawling
#### 2. Tranformation included Cleansing & Denoising
#### 3. Imputation

In [1]:
import sys
import ssl
from urllib.request import Request, urlopen
from datetime import datetime

def crawling(url='', encoding='utf-8', err=lambda e: print(f'{e} : {datetime.now()}', file=sys.stderr)):
    try:
        req = Request(url)
        ssl._create_default_https_context = ssl._create_unverified_context

        resp = urlopen(req)
        recv = resp.read().decode(encoding, errors='replace')

        print(f'{datetime.now()}: success for request [{url}]')
        return recv
    except Exception as e:
        err(e)

In [2]:
from functools import reduce
from itertools import count

resfetch = []
for idx in count(start=1):
    l = len(resfetch)
    resfetch[l:] = list(filter(
        lambda s: s.startswith('#'),
        reduce(
            lambda param, func: func(param),
            [lambda txt: [] if txt is None else txt.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
            crawling(
                'https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week0%02d.md' % idx,
                err=lambda e: None
            )
        )
    ))
    if len(resfetch) == l:
        break

print(resfetch)

2022-08-27 15:50:44.348075: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week001.md]
2022-08-27 15:50:44.961233: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week002.md]
2022-08-27 15:50:45.575920: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week003.md]
2022-08-27 15:50:46.195892: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week004.md]
2022-08-27 15:50:46.503649: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week005.md]
2022-08-27 15:50:47.111162: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week006.md]
2022-08-27 15:50:47.725382: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-reco

In [3]:
import re

restransform = []
for res in resfetch:
    if res.startswith('####'):
        name, record = (s.strip() for s in res.replace('####', '').strip().split(':'))
        # sval, unit = re.compile(r'(\d+.\d*)\s*(.*)').match(re.sub(r"[\([{})\]\s]", "", val)).groups()
        restransform[len(restransform):] = [(date, re.sub(r'[\d+\\.]', '', name.strip()).strip().upper(), *re.split(r'\s+', record.strip()))]
    elif res.startswith('##'):
        date = res.replace('##', '').strip()

print(restransform)

[('2021/06/24', 'FLAT BENCH PRESS[MACHINE, SMITH]', '40kg', '12reps', '5sets'), ('2021/06/24', 'SEATED CHEST PRESS[MACHINE]', '20kg', '12reps', '5sets'), ('2021/06/24', 'LAT PULLDOWN[MACHINE]', '20kg', '12reps', '5sets'), ('2021/06/24', 'SEATED LOW ROW[CABLE]', '15kg', '12reps', '3sets'), ('2021/06/24', 'ASSISTED PULLUP[MACHINE]', '-50kg', '12reps', '3sets'), ('2021/06/24', 'SHOULDER PRESS[MACHINE]', '5kg', '12reps', '5sets'), ('2021/06/25', 'LOW PULLEY[MACHINE]', '20kg', '12reps', '5sets'), ('2021/06/25', 'ASSISTED PULLUP[MACHINE]', '-50kg', '12reps', '5sets'), ('2021/06/25', 'SHOULDER PRESS[MACHINE]', '15kg', '12reps', '5sets(5th', 'failed)'), ('2021/06/25', 'LEG EXTENSION[MACHINE]', '20kg', '12reps', '3sets,', '25kg', '12reps', '2sets,', '30kg', '10reps', '2sets'), ('2021/06/25', 'LYING LEG CURLS[MACHINE]', '20kg', '12reps', '5sets(5th', 'failed)'), ('2021/06/25', 'SIDE LATERAL RAISE[DUMBBELL]', '4kg', '12reps', '5sets'), ('2021/06/28', 'FLAT BENCH PRESS[MACHINE, SMITH]', '40kg', '1

In [4]:
# Which Exercise?

exs = [t[1] for t in restransform]
dataset_exs = [(ex, exs.count(ex)) for ex in list(set(exs))]

sorted_by_ex = sorted(dataset_exs, key=lambda t: t[0])
for d in sorted_by_ex:
    print(d)

('ARNOLD PRESS[DUMBBELL]', 16)
('ASSISTED DIPS[MACHINE]', 24)
('ASSISTED DIP[MACHINE]', 14)
('ASSISTED PULLUP[MACHINE]', 55)
('BACK EXTENSION', 36)
('BASIC SQUAT', 4)
('BASIC SQUATS', 1)
('BEHIND THE NECK PRESS[MACHINE, SMITH]', 3)
('BENT OVER LATERAL RAISE[DUMBBELL]', 2)
('BENT OVER ROW[BARBELL]', 5)
('BENT OVER ROW[DUMBBELL ]', 1)
('BENT OVER ROW[DUMBBELL]', 13)
('BICEPS CURLS [EZBAR]', 1)
('BICEPS CURLS [FIXED BARBELL]', 1)
('BICEPS CURLS [MACHINE]', 13)
('CABLE CHEST FLY[CABLE]', 1)
('CABLE CROSS-OVER[CABLE]', 4)
('CABLE TRICEPS PUSHDOWN[CABLE]', 8)
('CHEST FLY [MACHINE, PEC DECK]', 2)
('CHEST FLY[MACHINE, PEC DECK]', 37)
('CHEST PRESS[MACHINE]', 2)
('CROSS LUNGES', 1)
('DEAD BUG', 2)
('DEADLIFT[BARBELL]', 83)
('DEADLIFT[DUMBBELL]', 8)
('DEADLIFT[KETTLEBELL]', 12)
('DECLINE BENCH PRESS[BARBELL]', 10)
('DIPS', 18)
('FACE PULLS[CABLE]', 1)
('FLAT BENCH PRESS[BARBELL]', 52)
('FLAT BENCH PRESS[DUMBBELL]', 2)
('FLAT BENCH PRESS[DUMBBEL]', 1)
('FLAT BENCH PRESS[FREE]', 1)
('FLAT BENCH PR

In [5]:
# and How Often?
sorted_by_ex = sorted(dataset_exs, key=lambda t: t[1], reverse=True)
for d in sorted_by_ex:
    print(d)

('LYING LEG CURLS[MACHINE]', 111)
('LAT PULLDOWN[MACHINE]', 98)
('PLANK', 92)
('SEATED CHEST PRESS[MACHINE]', 86)
('DEADLIFT[BARBELL]', 83)
('HIP ADDUCTION[MACHINE]', 81)
('SHOULDER PRESS[MACHINE]', 73)
('HIP ABDUCTION[MACHINE]', 73)
('SEATED ROW[MACHINE]', 62)
('LEG EXTENSION[MACHINE]', 62)
('SIDE LATERAL RAISE[DUMBBELL]', 60)
('ASSISTED PULLUP[MACHINE]', 55)
('FLAT BENCH PRESS[BARBELL]', 52)
('POWER LEG PRESS[MACHINE]', 50)
('STRAIGHT ARM PULLDOWN[CABLE]', 39)
('CHEST FLY[MACHINE, PEC DECK]', 37)
('ONE LEG DEADLIFT[KETTLEBELL]', 36)
('BACK EXTENSION', 36)
('WIDE PULLDOWN REAR[MACHINE]', 33)
('SEATED LOW ROW[CABLE]', 33)
('RUNNING', 33)
('SIDE-LYING HIP ABDUCTION', 32)
('INCLINE BENCH PRESS[MACHINE, SMITH]', 25)
('ASSISTED DIPS[MACHINE]', 24)
('WRIST ROLLER', 22)
('LONG PULL ROW[CABLE]', 19)
('INCLINE CHEST PRESS[MACHINE]', 19)
('DIPS', 18)
('INCLINE BENCH PRESS[BARBELL]', 17)
('REVERSE PEC DECK FLY[MACHINE, PEC DECK]', 16)
('ARNOLD PRESS[DUMBBELL]', 16)
('INCLINE BENCH PRESS[MACHINE]